In [12]:
import pandas as pd

from galaxy_mnist import GalaxyMNIST
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt

In [3]:
# Define transformation for the dataset
transform = transforms.Compose([transforms.ToTensor()])

# Load the GalaxyMNIST dataset
train_dataset = GalaxyMNIST(
    root='ML_DP/gal_mnist',
    download=True,
    train=True  # by default, or False for canonical test set
)
test_dataset = GalaxyMNIST(
    root='ML_DP/gal_mnist',
    download=True,
    train=False  # by default, or False for canonical test set
)

# images (as tensors) and labels
train_images = (train_dataset.data)
train_label = (train_dataset.targets)

test_images = (test_dataset.data)
test_label = (test_dataset.targets)

# create pytorch dataset
train_dataset = TensorDataset(train_images, train_label)
test_dataset = TensorDataset(test_images, test_label)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [4]:
# Define the CNN model
class GalaxyCNN(nn.Module):
    def __init__(self):
        super(GalaxyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 4)  # 4 classes for classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
# Instantiate the model, loss function, and optimizer
model = GalaxyCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

train_loss = np.zeros(num_epochs)

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.float()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    train_loss[epoch] = loss
    print(train_loss[epoch])
    # Print training loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

0.7640277147293091
Epoch [1/10], Loss: 0.7640
0.5413082242012024
Epoch [2/10], Loss: 0.5413
0.4992194175720215
Epoch [3/10], Loss: 0.4992
0.4143455922603607
Epoch [4/10], Loss: 0.4143
0.8527770042419434
Epoch [5/10], Loss: 0.8528
0.28456756472587585
Epoch [6/10], Loss: 0.2846
0.4067787528038025
Epoch [7/10], Loss: 0.4068
0.2947241961956024
Epoch [8/10], Loss: 0.2947
0.2844594419002533
Epoch [9/10], Loss: 0.2845
0.19354501366615295
Epoch [10/10], Loss: 0.1935


In [21]:
# Evaluation
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.float()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 74.05%
